In [1]:
#Omid55

Imports

In [3]:
import numpy as np

Functions

In [4]:
def unique_rows(A, return_index=False, return_inverse=False):
    """
    Similar to MATLAB's unique(A, 'rows'), this returns B, I, J
    where B is the unique rows of A and I and J satisfy
    A = B[J,:] and B = A[I,:]

    Returns I if return_index is True
    Returns J if return_inverse is True
    """
    A = np.require(A, requirements='C')
    assert A.ndim == 2, "array must be 2-dim'l"

    B = np.unique(A.view([('', A.dtype)]*A.shape[1]),
               return_index=return_index,
               return_inverse=return_inverse)

    if return_index or return_inverse:
        return (B[0].view(A.dtype).reshape((-1, A.shape[1]), order='C'),) \
            + B[1:]
    else:
        return B.view(A.dtype).reshape((-1, A.shape[1]), order='C')

In [2]:
def clean_it(dataset):
    TEAM_SIZE = 5
    # remove those teams with 3 members
    print dataset.shape
    dataset = np.delete(dataset,np.where(dataset == -1)[0],axis=0) 
    print dataset.shape
    
    # remove identical matches
    dataset = unique_rows(dataset)
    print dataset.shape
    
    # remove identical teams
    if dataset.shape[1] <= 2*TEAM_SIZE + 1:
        # just champion levels
        dataset = np.delete(dataset, np.where(np.sum(dataset[:,:TEAM_SIZE] - dataset[:,TEAM_SIZE:2*TEAM_SIZE],axis=1)==0),axis=0)
    else:
        # champion levels and members
        dataset = np.delete(dataset, np.where(np.sum(dataset[:,:TEAM_SIZE] - dataset[:,TEAM_SIZE:2*TEAM_SIZE] + 
          dataset[:,2*TEAM_SIZE:3*TEAM_SIZE] - dataset[:,3*TEAM_SIZE:-1], axis=1) == 0), axis=0)
    print dataset.shape
    
    return dataset

In [5]:
def compute_and_show_difference_of_winners_from_losers(matches):
    diff = np.zeros(len(matches))
    for i, match in enumerate(matches):
        loser = match[:TEAM_SIZE]
        winner = match[TEAM_SIZE:]
        diff[i] = LA.norm(winner,2) - LA.norm(loser,2)
        
    sns.set(rc={"figure.figsize": (20, 10)})
    
    ax = sns.distplot(diff)
    ax.set(xlabel='norm2(winner) - norm2(loser)', ylabel='Volume')
    
    sns.distplot(diff, fit=norm, kde=True)
    sns.plt.show()
    
    plt.hist(diff, bins=30)
    plt.xlabel("norm2(winner) - norm2(loser)")
    plt.ylabel("Volume")
    fig = plt.gcf()
    
    print 'L2-difference positive:', 100*float(len(np.where(diff > 0)[0]))/len(diff)
    print 'L2-difference zero:', 100*float(len(np.where(diff == 0)[0]))/len(diff)
    print 'L2-difference negative: ', float(100*len(np.where(diff < 0)[0]))/len(diff)
    
    return diff

In [6]:
def save_unique_difference(matches, name):
    X = matches[:,TEAM_SIZE:] - matches[:,:TEAM_SIZE]    #diff = winner - loser
    print(X.shape)
    X = np.delete(X, np.where(np.sum(X, axis=1) == 0), axis=0)
    print(X.shape)
    X = unique_rows(X)
    print(X.shape)

    np.savetxt(
        name,
        X,
        fmt='%d',
        delimiter=',',
        newline='\n',  # new line character
        footer='',  # file footer
        comments='',  # character to use for comments
        header='winner1_loser1,winner2_loser2,winner3_loser3,winner4_loser4,winner5_loser5'
        )
    
    print 'L1-difference positive: ', 100*len(np.where(np.sum(X, axis=1)>0)[0]) / len(X)
    print 'L1-difference zero: ', 100*len(np.where(np.sum(X, axis=1)==0)[0]) / len(X)
    print 'L1-difference negative: ', 100*len(np.where(np.sum(X, axis=1)<0)[0]) / len(X)
    
    return X

Body

In [ ]:
'''Loading the data'''
matches = np.genfromtxt('matches_reduced.csv', delimiter=',', skip_header=True)
TEAM_SIZE = 5
members = np.genfromtxt('members_in_matches_reduced.csv', delimiter=',', skip_header=True)

matches = clean_it(matches)